## Basic Setup

In [ ]:
import numpy as np
import zipfile
from datasets import Dataset
import pandas as pd

import os
# run as much as possible accelerated by apple silicon, fall back to cpu if not possible
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"


import sys
sys.path.append('..')
from scripts.audio import audio_file_to_array
from scripts.ipa import timit2ipa
from scripts.ipa import filter_chars

In [2]:
import sys, os
sys.path.append('..')
from scripts.eval_tests.panphon_model_eval import panphon_model_eval 

In [3]:
# set espeak library path for macOS
if sys.platform == "darwin":
    from phonemizer.backend.espeak.wrapper import EspeakWrapper

    _ESPEAK_LIBRARY = "/opt/homebrew/Cellar/espeak/1.48.04_1/lib/libespeak.1.1.48.dylib"
    EspeakWrapper.set_library(_ESPEAK_LIBRARY)

In [4]:
from transformers import AutoProcessor, AutoModelForCTC, AutoTokenizer, AutoFeatureExtractor

In [5]:
OUTPUT_DIR = os.path.join('..', 'models', 'timit-xlsr-finetune-B')
PRE_TRAINED_ID = "ginic/gender_split_70_female_4_wav2vec2-large-xlsr-53-buckeye-ipa"

# Load Data

In [6]:
timit = zipfile.ZipFile('../.data/TIMIT.zip', 'r')
timit_files = timit.namelist()

In [7]:
training_files = list(set(map(lambda x: x.split('.')[0], filter(lambda x: x.startswith('data/TRAIN'), timit_files))))

In [8]:
WAV_HEADER_SIZE = 44
def zipped_wav_to_array(filename):
    with timit.open(filename) as wav_file:
        return np.frombuffer(wav_file.read(), dtype=np.int16)[WAV_HEADER_SIZE//2:]

## Clean Up Dataset Vocab 
You will see that the initial vocab of most multilingual phoneme transcription models is quite big (300-400 phonemes) we will reduce this in the dataset to improve overall performance for English

In [ ]:
# First, ensure that the simplify_timit function is available and works
def simplify_timit(phoneme):
    # Substitute the phoneme based on the dictionary
    substitution_dict = {
        'ɾ̃': 'ɾ',  # Replace nasalized flap with plain flap
        'ŋ̍': 'ŋ',  # Remove syllabic marker from 'ŋ̍'
        'ə̥': 'ə',  # Remove voiceless marker from 'ə̥'
        'ɝ': 'ɹ',   # Simplify rhotacized schwa to 'ɹ'
        'ɚ': 'ɹ',   # Simplify rhotacized schwa to 'ɹ'
        'l̩': 'l',   # Remove syllabic marker from 'l̩'
        'm̩': 'm',   # Remove syllabic marker from 'm̩'
        'n̩': 'n',   # Remove syllabic marker from 'n̩'
        '̩': '',     # Remove syllabic marker
        'ʉ': 'u',    # Replace high central rounded vowel with high back rounded vowel
        'ɨ': 'i',    # Replace high central unrounded vowel with high front unrounded vowel
        ' ': '',     # Remove nasalization marker
        'ɦ': 'h',    # Replace voiceless glottal fricative with voiceless glottal fricative
        # Add other necessary substitutions if needed
    }
    # Apply the substitution for the phoneme
    return substitution_dict.get(phoneme, phoneme)  # Return simplified phoneme or the original if no replacement

def remove_stress_mark(text):
    """
    Removes the combining double inverted breve (͡) from text.
    
    Args:
        text: String or iterable containing IPA symbols
        
    Returns:
        Text with stress marks removed
    """
    if isinstance(text, str):
        return text.replace('͡', '')
    else:
        raise TypeError("Input must be string, set, or list")
    
# Updated timit_file_to_dict to use the simplified IPA phonemes
def timit_file_to_dict(filename):
    with timit.open(filename + '.PHN') as phn_file:
        timestamped_phonemes = []
        for line in phn_file.read().decode('utf-8').split('\n'):
            if line == '':
                continue
            start, end, phoneme = line.split()
            
            # Convert to IPA first and then simplify the phoneme
            ipa_phonemes = timit2ipa(phoneme, "eng")
            
            # Now filter only the necessary characters (this simplifies the phoneme to the basic form)
            cleaned_ipa = filter_chars(simplify_timit(ipa_phonemes), filter_type="letters")
            cleaned_ipa = remove_stress_mark(cleaned_ipa)
            timestamped_phonemes.append((cleaned_ipa, int(start), int(end)))

    return {'timestamped_phonemes': timestamped_phonemes, 'wav_filename': filename + '.WAV'}

# Updated files_to_df to create DataFrame with cleaned-up phonemes
def files_to_df(files):
    records = []
    for filename in files:
        parsed = timit_file_to_dict(filename)
        parsed['audio'] = zipped_wav_to_array(parsed['wav_filename'])
        del parsed['wav_filename']
        parsed['ipa'] = "".join(phoneme for phoneme, _, _ in parsed['timestamped_phonemes'])
        parsed['phoneme_starts'] = [start for _, start, _ in parsed['timestamped_phonemes']]
        parsed['phoneme_ends'] = [end for _, _, end in parsed['timestamped_phonemes']]
        del parsed['timestamped_phonemes']
        records.append(parsed)
    return pd.DataFrame(records)

# Call files_to_df for your test files
train_df = files_to_df(training_files)
train_df.head()


### Cleaning Up Audio
We will just crop out the start signal :)

In [10]:
def crop_audio(row):
    # Access the phoneme_starts column and get the last value of the list
    end = row['phoneme_starts'][-1]
    # Access the phoneme_ends column and get the first value of the list
    start = row['phoneme_ends'][0]
    # Crop the audio from start to end
    # note that start and end are in samples, not seconds
    cropped_audio = row['audio'][start:end]
    
    return cropped_audio

In [ ]:
# graph one audio to see start signal that we will remove
import matplotlib.pyplot as plt
plt.plot(train_df['audio'][0])
plt.show()

In [ ]:
# Apply the process_row function to each row in the DataFrame
train_df['cropped_audio'] = train_df.apply(crop_audio, axis=1) 
train_df.head()
    

In [ ]:
# visualize again, much better!
plt.plot(train_df['cropped_audio'][0])


In [ ]:
# train_df comprised of just cropped_audio and phonemes
train_df = train_df.drop(columns=['phoneme_starts', 'phoneme_ends', 'audio'])
# rename cropped audio to audio
train_df = train_df.rename(columns={'cropped_audio': 'audio'})
train_df.head()

In [15]:
train_ds = Dataset.from_pandas(train_df)

# Extend Phoneme Vocab

In [ ]:
timit_vocab = set("".join(train_df['ipa']))
print(timit_vocab)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_ID)
vocab = tokenizer.get_vocab()
# you will see how large the vocab is, we will resize our linear layer later to make it work for our smaller vocab
print(vocab)

In [ ]:
additional_vocab = timit_vocab.difference(set(vocab.keys()) | {' '})
print("tokens that are in timit but not in the pretrained model", additional_vocab)
tokenizer.add_tokens(list(additional_vocab))

# Update Feature Extraction

In [19]:
feature_extractor = AutoFeatureExtractor.from_pretrained(PRE_TRAINED_ID)

We don't need to update the feature extractor since it has been pretrained on 16kHz audio which matches the TIMIT dataset.

For datasets with different sampling rates, the feature extractor should be updated or the audio resampled (easier).

This is also where code to add extra features (such as conditioning on speaker's native language etc.) would be added.

# hyperparam search


In [ ]:
%pip install torchaudio

## v2 debug: changing handling of special tokens 

In [ ]:
import json
import torch
import logging
import tempfile
import wandb
import numpy as np
from transformers import (
    AutoProcessor, 
    AutoModelForCTC, 
    Trainer, 
    TrainingArguments, 
    Wav2Vec2CTCTokenizer,
    EarlyStoppingCallback,
    TrainerCallback
)
from datasets import Dataset
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torchaudio
from sklearn.model_selection import train_test_split
from itertools import product
from datetime import datetime
import time
import GPUtil
import psutil

class TrialTimeTracker:
    """Track detailed timing information for each trial"""
    def __init__(self, output_file):
        self.output_file = output_file
        self.current_trial = None
        self.start_time = None
        
    def start_trial(self, trial_config, trial_id):
        self.current_trial = {
            'trial_id': trial_id,
            'config': trial_config,
            'start_time': datetime.now().isoformat(),
            'gpu_type': self._get_gpu_type(),
            'timestamps': []
        }
        self.start_time = time.time()
        
    def log_timestamp(self, step_name):
        if self.current_trial:
            gpu_metrics = self._get_gpu_metrics()
            timestamp = {
                'step': step_name,
                'elapsed_seconds': time.time() - self.start_time,
                **gpu_metrics
            }
            self.current_trial['timestamps'].append(timestamp)
            logger.info(f"Step {step_name} completed after {timestamp['elapsed_seconds']:.2f}s")
            
    def end_trial(self, eval_loss=None):
        if self.current_trial:
            self.current_trial['total_time'] = time.time() - self.start_time
            if eval_loss is not None:
                self.current_trial['eval_loss'] = eval_loss
            
            with open(self.output_file, 'a') as f:
                json.dump(self.current_trial, f)
                f.write('\n')
            
            trial_summary = self.current_trial.copy()
            self.current_trial = None
            return trial_summary
    
    def _get_gpu_type(self):
        try:
            gpu = GPUtil.getGPUs()[0]
            return gpu.name
        except:
            return "Unknown"
            
    def _get_gpu_metrics(self):
        try:
            gpu = GPUtil.getGPUs()[0]
            return {
                'gpu_utilization': gpu.load * 100,
                'gpu_memory_used': gpu.memoryUsed,
                'gpu_memory_total': gpu.memoryTotal
            }
        except:
            return {
                'gpu_utilization': None,
                'gpu_memory_used': None,
                'gpu_memory_total': None
            }

# Constants
SAMPLING_RATE = 16000
MAX_AUDIO_LENGTH = 160000  # 10 seconds at 16kHz
MAX_LABEL_LENGTH = 100

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    """
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        max_length = min(
            max(len(feature["input_values"]) for feature in features),
            MAX_AUDIO_LENGTH
        )
        padded_inputs = []
        attention_mask = []
        for feature in features:
            input_length = len(feature["input_values"])
            padding_length = max_length - input_length
            
            if isinstance(feature["input_values"], list):
                input_values = torch.tensor(feature["input_values"])
            else:
                input_values = feature["input_values"]
            
            input_values = input_values.squeeze()
            
            if padding_length > 0:
                padded_input = torch.nn.functional.pad(input_values, (0, padding_length))
                attention_mask.append(torch.cat([torch.ones(input_length), torch.zeros(padding_length)]))
            else:
                padded_input = input_values
                attention_mask.append(torch.ones(input_length))
            
            padded_inputs.append(padded_input)

        batch = {
            "input_values": torch.stack(padded_inputs),
            "attention_mask": torch.stack(attention_mask)
        }

        with self.processor.as_target_processor():
            label_features = [{"input_ids": feature["labels"]} for feature in features]
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch

def prepare_datasets(train_df, processor):
    """
    Dataset preparation with optimized batch processing
    """
    logger.info("Splitting dataset into train and evaluation...")
    
    train_data, eval_data = train_test_split(train_df, test_size=0.2, random_state=42)
    
    def process_data(batch):
        # Preprocess audio
        audio = preprocess_audio(batch["audio"])
        audio = audio.squeeze()
        
        # Process audio without padding or tensor conversion
        inputs = processor(
            audio, 
            sampling_rate=SAMPLING_RATE, 
            return_tensors=None
        )
        
        input_values = np.squeeze(inputs["input_values"])
        
        # Process labels
        with processor.as_target_processor():
            labels = processor(batch["ipa"]).input_ids
        
        return {
            "input_values": input_values,
            "labels": labels,
            "audio": audio
        }
    
    # Convert DataFrames to Datasets with optimized batch processing
    train_dataset = Dataset.from_pandas(train_data)
    eval_dataset = Dataset.from_pandas(eval_data)
    
    # Apply processing with larger batches
    train_dataset = train_dataset.map(
        process_data,
        batch_size=32,
        num_proc=4,
        remove_columns=[col for col in train_dataset.column_names if col != "audio"]
    )
    eval_dataset = eval_dataset.map(
        process_data,
        batch_size=32,
        num_proc=4,
        remove_columns=[col for col in eval_dataset.column_names if col != "audio"]
    )
    
    return train_dataset, eval_dataset

def preprocess_audio(audio_input):
    """
    Load and preprocess audio to match expected format.
    """
    # Handle different input types
    if isinstance(audio_input, str):  # File path
        waveform, sample_rate = torchaudio.load(audio_input)
        audio_numpy = waveform.squeeze().numpy()
        
    elif isinstance(audio_input, list):  # List input
        audio_numpy = np.array(audio_input, dtype=np.float32)
        
    elif isinstance(audio_input, np.ndarray):  # Already numpy array
        audio_numpy = audio_input
        
    elif isinstance(audio_input, torch.Tensor):  # Torch tensor
        audio_numpy = audio_input.numpy()
        
    else:
        raise ValueError(f"Unsupported audio type: {type(audio_input)}")
    
    # Ensure float32 dtype
    audio_numpy = audio_numpy.astype(np.float32)
    
    # Ensure 1D array
    if len(audio_numpy.shape) > 1:
        audio_numpy = np.mean(audio_numpy, axis=0)
    
    # Normalize audio
    if np.abs(audio_numpy).max() > 1:
        audio_numpy = audio_numpy / np.abs(audio_numpy).max()
    
    # Trim or pad to max length
    if len(audio_numpy) > MAX_AUDIO_LENGTH:
        audio_numpy = audio_numpy[:MAX_AUDIO_LENGTH]
    elif len(audio_numpy) < MAX_AUDIO_LENGTH:
        padding = np.zeros(MAX_AUDIO_LENGTH - len(audio_numpy), dtype=np.float32)
        audio_numpy = np.concatenate([audio_numpy, padding])
    
    return audio_numpy

def prepare_model_and_processor(pre_trained_id, timit_vocab):
    """
    Prepare the model and processor with reduced vocabulary and resized linear head.
    """
    logger.info("Loading pretrained model and processor...")
    
    # Create base directory
    base_dir = "/home/arunasri/ML/notebooks/results/a100_resize_structured_v4"
    os.makedirs(base_dir, exist_ok=True)
    
    processor = AutoProcessor.from_pretrained(pre_trained_id)
    model = AutoModelForCTC.from_pretrained(pre_trained_id)
    
    # Log initial vocabulary details
    old_vocab = processor.tokenizer.get_vocab()
    logger.info(f"Initial vocabulary size: {len(old_vocab)}")
    
    # Define special tokens with fixed positions matching pretrained model
    special_tokens = {
        "[PAD]": 0,
        "<s>": 1,
        "</s>": 2,
        "[UNK]": 3
    }
    
    # Create new vocabulary
    regular_tokens = sorted(list(timit_vocab - set(['<pad>', '<unk>'])))  # Remove old-style tokens if present
    vocab = {**special_tokens}
    
    for idx, token in enumerate(regular_tokens):
        vocab[token] = idx + len(special_tokens)
    
    logger.info(f"New vocabulary size: {len(vocab)}")
    
    # Save vocabulary for inspection
    vocab_file = os.path.join(base_dir, "vocab.json")
    with open(vocab_file, 'w') as f:
        json.dump(vocab, f, indent=2)
    
    # Create new tokenizer with matching special tokens
    tokenizer_config = {
        "vocab": vocab,
        "pad_token": "[PAD]",
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "[UNK]",
    }
    
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        json.dump(vocab, f)
        vocab_path = f.name
    
    new_tokenizer = Wav2Vec2CTCTokenizer(
        vocab_path,
        **{k: v for k, v in tokenizer_config.items() if k != "vocab"}
    )
    
    processor.tokenizer = new_tokenizer
    
    # Save processor configuration
    processor_path = os.path.join(base_dir, "processor")
    processor.save_pretrained(processor_path)
    logger.info(f"Saved processor configuration to {processor_path}")
    
    # Resize the linear head
    old_weights = model.lm_head.weight.data
    old_bias = model.lm_head.bias.data
    
    new_layer = torch.nn.Linear(model.lm_head.in_features, len(processor.tokenizer))
    torch.nn.init.xavier_uniform_(new_layer.weight.data, gain=0.1)
    new_layer.bias.data.uniform_(-0.1, 0.1)
    
    # Transfer weights for matching tokens
    transfer_count = 0
    for token, new_idx in vocab.items():
        if token in old_vocab:
            old_idx = old_vocab[token]
            new_layer.weight.data[new_idx, :] = old_weights[old_idx, :]
            new_layer.bias.data[new_idx] = old_bias[old_idx]
            transfer_count += 1
            logger.info(f"Transferred token '{token}' from position {old_idx} to {new_idx}")
    
    logger.info(f"Transferred weights for {transfer_count} tokens")
    
    # Update model configuration
    model.lm_head = new_layer
    model.config.vocab_size = len(vocab)
    model.config.pad_token_id = special_tokens["[PAD]"]
    model.config.bos_token_id = special_tokens["<s>"]
    model.config.eos_token_id = special_tokens["</s>"]
    model.config.unk_token_id = special_tokens["[UNK]"]
    
    return model, processor

def generate_systematic_configs():
    """Generate systematic hyperparameter configurations"""
    learning_rates = np.logspace(-6, -4, num=5)
    batch_sizes = [4, 8, 16]
    grad_accums = [2, 4, 8]
    warmup_ratios = [0.05, 0.1, 0.15]
    weight_decays = np.logspace(-3, -1, num=3)
    max_grad_norms = [0.5, 1.0, 2.0]
    adam_beta1s = [0.85, 0.9, 0.95]
    adam_beta2s = [0.95, 0.975, 0.999]
    adam_epsilons = np.logspace(-9, -7, num=3)
    
    primary_configs = list(product(
        learning_rates,
        batch_sizes,
        grad_accums
    ))
    
    secondary_params = {
        'warmup_ratio': warmup_ratios[1],
        'weight_decay': weight_decays[1],
        'max_grad_norm': max_grad_norms[1],
        'adam_beta1': adam_beta1s[1],
        'adam_beta2': adam_beta2s[1],
        'adam_epsilon': adam_epsilons[1]
    }
    
    configs = []
    for lr, bs, ga in primary_configs:
        config = {
            'learning_rate': float(lr),
            'per_device_train_batch_size': bs,
            'per_device_eval_batch_size': bs,
            'gradient_accumulation_steps': ga,
            **secondary_params
        }
        configs.append(config)
    
    return configs
class LossMonitorCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            if 'loss' in logs:
                current_loss = logs['loss']
                if current_loss > 10.0:
                    logger.warning(f"High loss detected: {current_loss}")
                    logger.warning("This may indicate issues with vocabulary handling or model configuration")
                elif current_loss < 0.1:
                    logger.warning(f"Unusually low loss detected: {current_loss}")
                    logger.warning("This may indicate potential issues with loss computation")

def run_systematic_search(train_df, pre_trained_id, timit_vocab, resume_file="a100_resize_structured_search_v4.jsonl", start_trial=0):
    """Run systematic hyperparameter search with enhanced monitoring"""
    configs = generate_systematic_configs()
    logger.info(f"Generated {len(configs)} configurations for systematic search")
    logger.info(f"Starting from trial {start_trial}")
    
    # Initialize tracking file - but only truncate if starting from beginning
    if start_trial == 0:
        tracker = TrialTimeTracker(output_file=resume_file)
        logger.info(f"Starting fresh search with results saved to {resume_file}")
    else:
        # Append mode for resuming
        tracker = TrialTimeTracker(output_file=resume_file)
        logger.info(f"Resuming search from trial {start_trial}, appending to {resume_file}")
    
    # Skip trials before start_trial
    for trial_idx, config in enumerate(configs[start_trial:], start=start_trial):
        logger.info(f"\nStarting trial {trial_idx + 1}/{len(configs)}")
        logger.info(f"Configuration: {json.dumps(config, indent=2)}")
        
        run_name = f"trial_{trial_idx}_resize_v4"
        wandb.init(
            project="xlsr-buckeye-phoneme-a100-resize-structured-v4-tokenizer",
            name=run_name,
            config=config,
            reinit=True
        )
        
        try:
            tracker.start_trial(config, trial_idx)
            
            # Initialize model and processor with validation
            tracker.log_timestamp("model_init_start")
            model, processor = prepare_model_and_processor(pre_trained_id, timit_vocab)
            model.gradient_checkpointing_enable()
            tracker.log_timestamp("model_init_complete")
            
            # Prepare datasets
            tracker.log_timestamp("dataset_prep_start")
            train_dataset, eval_dataset = prepare_datasets(train_df, processor)
            tracker.log_timestamp("dataset_prep_complete")
            
            # Training setup
            output_dir = f"./results/a100_resize_structured_v4_tokenizer/trial_{trial_idx}"
            training_args = TrainingArguments(
                output_dir=output_dir,
                **config,
                num_train_epochs=5,
                logging_steps=50,
                save_steps=100,
                eval_steps=100,
                evaluation_strategy="steps",
                load_best_model_at_end=True,
                metric_for_best_model="eval_loss",
                greater_is_better=False,
                save_strategy="steps",
                save_total_limit=2,
                fp16=True,
                dataloader_num_workers=16,
                gradient_checkpointing=True,
                dataloader_prefetch_factor=4,
                ddp_find_unused_parameters=False,
                resume_from_checkpoint=True
            )
            
            # Initialize trainer with callbacks
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
                data_collator=DataCollatorCTCWithPadding(processor=processor, padding="longest"),
                callbacks=[
                    EarlyStoppingCallback(early_stopping_patience=3),
                    LossMonitorCallback()
                ]
            )
            
            # Training
            tracker.log_timestamp("training_start")
            train_result = trainer.train()
            tracker.log_timestamp("training_complete")
            
            # Evaluation
            eval_result = trainer.evaluate()
            tracker.log_timestamp("evaluation_complete")
            
            # Log results
            wandb.log({
                "eval_loss": eval_result["eval_loss"],
                "train_loss": train_result.training_loss,
                "total_training_time": train_result.metrics["train_runtime"],
                "samples_per_second": train_result.metrics["train_samples_per_second"]
            })
            
            tracker.end_trial(eval_loss=eval_result["eval_loss"])
            
        except Exception as e:
            logger.error(f"Error in trial {trial_idx}: {str(e)}")
            tracker.end_trial()
        
        finally:
            wandb.finish()
    
    return resume_file

if __name__ == "__main__":
    results_file = run_systematic_search(
        train_df=train_df,
        pre_trained_id=PRE_TRAINED_ID,
        timit_vocab=timit_vocab,
        start_trial=15
    )
    logger.info(f"Results saved to {results_file}")

## Save Model on Hugging Face

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor
from huggingface_hub import login

# Log in to Hugging Face (if not already logged in)
# login("your-huggingface-api-token")

# Load the model and processor from your local directory
model = AutoModelForCTC.from_pretrained("/home/arunasrivastava/ML/notebooks/results-b0")
processor = Wav2Vec2Processor.from_pretrained("/home/arunasrivastava/ML/notebooks/results-b0")

# Specify the directory where you want to save the model
save_directory = "./xlsr-timit-b0"

# Save and upload to Hugging Face Model Hub under an organization
model.save_pretrained(save_directory, push_to_hub=True, repo_id="KoelLabs/xlsr-timit-b0")
processor.save_pretrained(save_directory)


# Instructions to Upload Model to Hugging Face

1. **Login to Hugging Face**  
   Run the following command to log in to your Hugging Face account:
   ```bash
   huggingface-cli login


2. **Create a Repository**  
   Run the following command to log in to your Hugging Face account:
   ```bash
   huggingface-cli repo create your-model-name --organization your-org-name

3. **Git clone**  
   Run the following command to clone your repo:
   ```bash 
   git clone https://huggingface.co/your-org-name/your-model-name

4. **Upload Model**  (if you did not make it on Hugging Face already)
   Run the following command to clone your repo:
   ```bash 
   huggingface-cli repo upload --path ./<path here>


5. **Move Other Files**  
   Run the following command to copy each file into the repo:
   ```bash
   cp -r ./<path here> <your-cloned-repo-folder> 

6. **Stage & Commit!**
    Run the following git commands
    ```bash
    git add README.md
    git commit -m "Add model card"
    git push


### YAAAAY CONGRATULATIONS 

# (Optional) Hyperparam search


In [ ]:
%pip install optuna joblib
# you can use WADNB sweeps as well, whatever is easier